# Alguns métodos de resolução de sistemas lineares

# Método de Gauss-Jordan

###### Neste método, transformamos a matriz de coeficientes em identidade; a matriz aumentada conterá a inversa da matriz de coeficientes, que utilizaremos para obter a solução do sistema. Pode-se inserir uma etapa de pivotação parcial ou total no processo.

In [1]:
%%latex
\begin{equation}
A*X = B \rightarrow X = A^{-1}*B
\end{equation}

<IPython.core.display.Latex object>

### Vamos começar com funções separadas para facilitar o entendimento e depois juntamos tudo numa maior.

In [2]:
import numpy as np
import pandas as pd

In [3]:
n = 3 ## tamanho da matriz de exemplo

In [4]:
M = np.zeros((n,2*n)) ##seta uma matriz de exemplo; a coluna neste método tem que ter dimensão 2n pois a matriz aumentada recebe a matriz identidade que vai constituir a inversa

In [5]:
id = np.identity(n) ##criamos a matriz identidade

In [6]:
M[:,n:2*n] = id #alocamos na matriz aumentada

In [7]:
M

array([[0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [8]:
###preenchendo com os coeficientes
M[0][0] = 3
M[0][1] = 1.5
M[0][2] = 4.75
M[1][0] = 4
M[1][1] = 2
M[1][2] = 3
M[2][0] = 2
M[2][1] = 5
M[2][2] = 3

In [26]:
### Vetor com as respostas:
b = np.array([8, 7, -12])

In [10]:
b

array([ 3.069, -4.288,  1.1  ])

### Pivotação parcial
#### A função abaixo vai encontrar o indice (de linha) do maior valor em módulo na coluna que queremos pivotar, a partir da linha i == j

In [11]:
def max_idx(M, n, j): ###
    max_ = 0
    idx = 0
    for i in range(j,n):
        if abs(M[i,j]) >= max_:
            max_ = M[i,j]
            idx = i
    return idx

### A função abaixo faz a troca de linhas, de acordo com a coluna j em que estamos

In [12]:
def pivota(M, n, j):
    idx_max = max_idx(M, n, j)
    temp = M[j,:].copy()
    M[j,:] = M[idx_max,:]
    M[idx_max,:] = temp
    return M

### Escalonamento da matriz

### Nesta etapa, não fazemos apenas a triangularização; temos que fazer a diagonalização completa da matriz para ela se tornar uma identidade. Logo, precisamos normalizar pelo pivô e alterar a função de triangularização para fazer diagonalização.

In [13]:
def normaliza(M, n, j):
    pivo = M[j, j]
    M[j,:] = M[j,:]/pivo
    return M

In [14]:
def diagonaliza(M, n, j):   ######esta função traz o indice da coluna em que estamos, a matriz, e o tamanho da matriz de coeficiente; ela vai diagonalizar a linha i fixando-se o indice da coluna j, para todo i != j 
    for i in range(n):
        if (i!=j):
            M[i,:] = M[i,:] - (M[i,j])*M[j,:]
    return M

### Juntando tudo numa funçãozinha só

In [15]:
###vamos juntar tudo numa função só para mostrar que funciona
def gauss_jordan(M,n):
    for j in range(n):
        M = pivota(M, n, j)
        M = normaliza(M, n, j)
        M = diagonaliza(M, n, j)
    return M

In [16]:
teste = gauss_jordan(M, n)

In [17]:
teste

array([[ 1.     ,  0.     ,  0.     , -0.225  ,  0.48125, -0.125  ],
       [ 0.     ,  1.     ,  0.     , -0.15   , -0.0125 ,  0.25   ],
       [ 0.     ,  0.     ,  1.     ,  0.4    , -0.3    ,  0.     ]])

### A implementação acima funciona, mas tem funçõezinhas em separado. Conseguimos juntar tudo em uma só: encontrar o maior valor em módulo, trocar as linhas, diagonalizar :)


In [18]:
def gauss_jordan_tot(M, n):
    for j in range(n):
        max_ = 0
        idx = 0
        for i in range(j,n):
            if abs(M[i,j]) >= max_:
                max_ = M[i,j]
                idx = i
        i=0
        temp = M[j,:].copy()
        M[j,:] = M[idx,:]
        M[idx,:] = temp
        pivo = M[j,j]
        M[j,:] = M[j,:]/pivo  ###normalizando
        for i in range(n):      ### ela vai diagonalizar a linha i fixando o indice da coluna, para i != j
            if (i!=j):
                M[i,:] = M[i,:] - (M[i,j])*M[j,:]
    return M

In [19]:
teste2 = gauss_jordan_tot(M, n)

In [20]:
teste2

array([[ 1.     ,  0.     ,  0.     , -0.225  ,  0.48125, -0.125  ],
       [ 0.     ,  1.     ,  0.     , -0.15   , -0.0125 ,  0.25   ],
       [ 0.     ,  0.     ,  1.     ,  0.4    , -0.3    ,  0.     ]])

## Atribuição das soluções do sistema

### Basta multiplicar a inversa pelo vetor de resposta do sistema.

In [22]:
M_1 = teste2[:,n:2*n+1]
M_1

array([[-0.225  ,  0.48125, -0.125  ],
       [-0.15   , -0.0125 ,  0.25   ],
       [ 0.4    , -0.3    ,  0.     ]])

In [23]:
pd.DataFrame(M_1)

,0,1,2
0,-0.225,0.48125,-0.125
1,-0.150,-0.01250,0.250
2,0.400,-0.30000,0.000


In [27]:
solucao = np.dot(M_1, b)
    

In [28]:
solucao

array([ 3.06875, -4.2875 ,  1.1    ])

### Verificando se resultou na solução correta:

In [34]:
###recarrega os coeficientes na M só para testarmos
M[0][0] = 3
M[0][1] = 1.5
M[0][2] = 4.75
M[1][0] = 4
M[1][1] = 2
M[1][2] = 3
M[2][0] = 2
M[2][1] = 5
M[2][2] = 3

In [35]:
np.dot(M[:,0:n],solucao) #Sim! Deu certo

array([  8.,   7., -12.])